In [7]:
import json
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer

In [8]:
sharegpt_ds = load_dataset("/data/lixubin/MiraTrainer/src/data/sft")

In [9]:
tokenizer = AutoTokenizer.from_pretrained('/data/lixubin/models/Qwen/Qwen1.5-7B-Chat/')
def trans_format(example):
    """
    an example in Vicuna format: 
        {
            "id": 1,
            "conversations": [
                { "role": "user", "content": "" },
                { "role": "assistant", "content": "" },
                { "role": "user", "content": "" },
                { "role": "assistant", "content": "" }
            ]
        }
    """
    history = example['history']+[example['instruction']+example['input'], example['output']]
    converstaions = []
    for pair in history:
        converstaions.append( {"role":"user",      "content":pair[0]} )
        converstaions.append( {"role":"assistant", "content":pair[1]} )

    return { "conversations": tokenizer.apply_chat_template(converstaions) }



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
sharegpt_ds['train']

Dataset({
    features: ['instruction', 'input', 'history', 'output'],
    num_rows: 147
})

In [11]:
vicuna_ds = sharegpt_ds.map(
    trans_format,
    remove_columns=['instruction', 'input', 'history', 'output']
)

Map: 100%|██████████| 147/147 [00:00<00:00, 1368.23 examples/s]


In [12]:
vicuna_ds

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 147
    })
})